In [1]:
import os
import paltas
from astropy.visualization import simple_norm
import visualization_utils
from paltas import generate
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

SMALL_SIZE = 16
MEDIUM_SIZE = 18
BIGGER_SIZE = 20


plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)    # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

%matplotlib inline
root_path = paltas.__path__[0][:-7]
import re

/Users/padmavenkatraman/Documents/StrongLensing/silver/lib/python3.11/site-packages/numba/core/decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)


In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline


### paltas PSF library

In [3]:
# psf_im = np.load('data/psf_images.npy', mmap_mode='r+')

In [4]:
# psf_im_i = psf_im[:, 1, :, :]

In [5]:
# # index = np.random.randint(0, psf_im_i.shape[0])
# index = 0
# psf_kernels = psf_im_i[index, :, :]
# psf_kernels[psf_kernels<0] = 0
# plt.imshow(psf_kernels);

In [6]:
# psf_sums = np.sum(psf_im_i,axis=(1,2))

In [7]:
# psf_sums = psf_sums.reshape(-1,1,1)

In [8]:
# normalized = psf_im_i/psf_sums
# normalized

In [9]:
# import tensorflow as tf
# normalized = tf.constant(normalized)
# pad_tns = tf.constant([[0, 0],[0,1],[0,1]])
# # -----------padding: ^first_dim^
# # ------------------------padding: ^second_dim^
# # ---------------------------------------padding: ^third_dim^

# # Generating padded Tensor
# res = tf.pad(normalized, pad_tns, mode ='CONSTANT', constant_values=0)
# print(res.shape)


In [10]:
# np.save('data/norm_resize_psf.npy', res)

In [11]:
# #plt.hist(normalized[0, :, 17], alpha=0.5)

# #plt.hist(normalized[0, :, 16], alpha = 0.5)
# plt.plot(normalized[0, :, 15], alpha=0.5, label=15)
# plt.plot(normalized[0, :, 16], alpha=0.5, label=16)
# plt.plot(normalized[0, :, 17], alpha=0.5, label=17)
# plt.plot(normalized[0, :, 18], alpha=0.5, label=18)
# plt.axvline(32//2)
# plt.legend()

In [12]:
# #plt.hist(normalized[0, :, 17], alpha=0.5)

# #plt.hist(normalized[0, :, 16], alpha = 0.5)
# plt.plot(normalized[0, 15, :], alpha=0.5, label=15)
# plt.plot(normalized[0, 16, :], alpha=0.5, label=16)
# plt.plot(normalized[0, 17, :], alpha=0.5, label=17)
# plt.plot(normalized[0, 18, :], alpha=0.5, label=18)
# plt.axvline(32//2)
# plt.legend()

In [13]:
# fig, ax = plt.subplots(1, 1, figsize=(6, 6))
# # ax[0].imshow(normalized_resized[0, :, :])
# ax.imshow(normalized[0, :, :])
# # ax[0].set_title('resized PSF: 31x31')
# ax.set_title('original PSF: 32x32')

In [14]:
# plt.hist(psf_sums.flatten(), density=True, bins=100);
# plt.title("PSF Sum Distribution")
# plt.xlabel("Sum of pixel in PSF kernel");

### Generate

In [ ]:
config_file = 'config_LSST'

In [ ]:
from paltas.Configs.config_handler import ConfigHandler

# We'll pass one of our example Configs in
config_handler = ConfigHandler(os.path.join(root_path,'paltas/notebooks/config_LSST.py'))

# First let's get the lenstronomy model lists, kwargs lists, and redshift lists.
kwargs_model, kwargs_params = config_handler.get_lenstronomy_models_kwargs()

# We can see what keys are returned, and inspect the lens models in particular
print('Available Keys:')
print(kwargs_model.keys())
print(kwargs_params.keys())
print('')

print('lens_model_list')
print(kwargs_model['lens_model_list'])
print('kwargs_lens')
print()
print(kwargs_params['kwargs_lens_light'])


In [ ]:
kwargs_params['kwargs_lens'][0]['theta_E']

In [ ]:
img, prop = config_handler.draw_image(new_sample=True)

In [ ]:
plt.imshow(img)

In [ ]:
# command = f'python3 ../paltas/generate.py {config_file}.py generated_images/test/{config_file} --n 1556 --tf_record'

In [ ]:
# os.system(command = command)
config_file

In [ ]:
command = f'python3 ../paltas/generate.py {config_file}.py generated_images/train_with_lens/{config_file} --n 100000 --tf_record --h5'

In [ ]:
os.system(command = command)


In [15]:
import paltas.Analysis.dataset_generation as dg

In [16]:
dg.generate_tf_record('generated_images/train_with_lens/config_LSST/',['main_deflector_parameters_theta_E',
	'main_deflector_parameters_gamma1','main_deflector_parameters_gamma2',
	'main_deflector_parameters_gamma','main_deflector_parameters_e1',
	'main_deflector_parameters_e2','main_deflector_parameters_center_x',
	'main_deflector_parameters_center_y'],'generated_images/train_with_lens/config_LSST/metadata.csv',
	'generated_images/train_with_lens/config_LSST/data.tfrecord',h5=True)

Saving 100000 files into the tf record


100%|██████████| 100000/100000 [00:05<00:00, 18338.31it/s]


In [ ]:
import paltas.Sampling.distributions as dist
from scipy.stats import norm, truncnorm, uniform
import matplotlib.pyplot as plt


In [ ]:

plt.hist(norm(loc=0,scale=0.5).rvs(size=1000))

In [ ]:
config_file_path = f'generated_images/no_lens/{config_file}/'

file_list = [os.path.join(config_file_path, i) for i in os.listdir(config_file_path)]
names = np.arange(100).astype(str)


In [ ]:
df = pd.read_csv(f'generated_images/no_lens/{config_file}/metadata.csv', index_col=0)

In [ ]:
start_index = np.random.randint(0, 100 - 24)
picked_files = file_list[start_index:start_index + 24]
picked_names = names[start_index:start_index + 24]
picked_names

In [ ]:
%matplotlib inline
visualization_utils.matrix_plot_from_npy(picked_files,names=picked_names,dim=(3,8),
                                         save_name='lsst.png',stretch = 'asinh',asinh_a = 0.1,
                                         annotate=False)
plt.show()
# visualization_utils.matrix_plot_from_npy(picked_files,names=picked_names,dim=(3,8),
#                                          save_name='lsst.png',asinh_a = 0.1, max_cut = 70, annotate=True)
# plt.show()

In [ ]:
# weights = sorted(os.listdir('train_results/model_weights'))
# losses = pd.read_csv('train_results/train.csv', index_col=0)
# file_list_bad= []
# num_files = len(weights)
# for i in range (num_files):
# # weights[0][-12:-8]
#     print(weights[i])
#     ind = int(weights[i][1:3])
#     ls = float(weights[i][-12:-8])
#     valloss = np.round(losses.loc[ind-1, 'val_loss'], 2)
#     # print(type(valloss), type(ls))
#     if not np.isclose(valloss,ls):
#         print(ind, 'latest loss: ', valloss, 'loss_on_file: ', ls)
#         file = f'train_results/model_weights/{weights[i]}'
#         file_list_bad.append(file)

In [ ]:
# losses = pd.read_csv('train_results/train.csv', index_col=0)
# epochs = len(losses['val_loss'])
# losses

In [ ]:
# losses = pd.read_csv('train_results/train.csv', index_col=0)
# epochs = losses.index
# plt.plot(epochs, losses['loss'], label='Training Loss')
# plt.plot(epochs, losses['val_loss'], label='Validation Loss')
# plt.legend()
# plt.title("Training Loss")

In [ ]:
os.system(command = f'python ../paltas/Analysis/train_model.py lsst_train.py --tensorboard_dir NLNORM')